In [2]:
import pandas as pd
import math
from scipy import stats
import xlsxwriter
import requests
from statistics import mean

In [4]:
stocks = pd.read_csv('/Users/User2/Documents/freecode camp algo trading/sp_500_stocks.csv')

IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

def chunks(list,n):
    for i in range(0,len(list),n):
        yield list[i:i+n]

sym_group = []
for i in chunks(stocks['Ticker'],100):
    sym_group.append(i)
sym_string = []
for i in sym_group:
    sym_string.append(','.join(i))
sym_string

my_columns = ['Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score']
df=pd.DataFrame(columns = my_columns)
for i in sym_string:
    api_url = batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={i}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    for symbol in i.split(','):
        df =df.append(pd.Series((symbol,data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   
                            ),index=my_columns),ignore_index=True)
    
df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,139.50,N/A,0.734487,N/A,0.276035,N/A,0.0688852,N/A,0.0995583,N/A,N/A
1,AAL,21.71,N/A,0.993043,N/A,0.769746,N/A,0.412879,N/A,-0.101884,N/A,N/A
2,AAP,207.37,N/A,0.660108,N/A,0.256632,N/A,0.171259,N/A,0.079956,N/A,N/A
3,AAPL,138.80,N/A,0.89879,N/A,0.136933,N/A,0.0586902,N/A,0.0705995,N/A,N/A
4,ABBV,110.80,N/A,0.38778,N/A,0.286825,N/A,-0.0118133,N/A,-0.0151112,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,120.96,N/A,0.517782,N/A,0.209568,N/A,0.112619,N/A,0.0878614,N/A,N/A
501,ZBH,181.64,N/A,0.608553,N/A,0.176968,N/A,0.075408,N/A,0.0773007,N/A,N/A
502,ZBRA,504.59,N/A,1.60356,N/A,0.772457,N/A,0.264562,N/A,0.0856231,N/A,N/A
503,ZION,53.95,N/A,1.18935,N/A,0.841498,N/A,0.152014,N/A,-0.00499451,N/A,N/A


In [4]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('please enter portfolio size:')
    
    try:
        val = float(portfolio_size)
    except ValueError:
        print ('please enter an integer \nplease try again')
        portfolio_input()
    
portfolio_input()



please enter portfolio size:1000000


In [7]:
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']

for row in df.index:
    for time_period in time_periods:
        if df.loc[row,f'{time_period} Price Return'] == None:
            df.loc[row,f'{time_period} Price Return'] =0

for row in df.index:
    for time_period in time_periods:
        df.loc[row,f'{time_period} Return Percentile']= stats.percentileofscore(df[f'{time_period} Price Return'], 
                                                                                df.loc[row,f'{time_period} Price Return'])
                                                                                                                                                                           

In [8]:
for row in df.index:
    mean_percentile = []
    for time_period in time_periods: 
        mean_percentile.append(df.loc[row,f'{time_period} Return Percentile'])
    df.loc[row,'HQM Score']=mean(mean_percentile)
    

df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,136.300,N/A,0.733783,59.0099,0.268661,51.8812,0.0688524,28.7129,0.0997759,83.1683,55.6931
1,AAL,22.280,N/A,1.03368,77.0297,0.79993,93.2673,0.408664,98.6139,-0.099906,2.77228,67.9208
2,AAP,201.982,N/A,0.672615,54.0594,0.24952,48.3168,0.173898,70.495,0.0814037,71.6832,61.1386
3,AAPL,135.090,N/A,0.923212,71.0891,0.133053,29.703,0.0581405,25.5446,0.0710166,66.7327,48.2673
4,ABBV,108.840,N/A,0.399993,31.6832,0.285951,54.8515,-0.0118227,7.32673,-0.014893,11.6832,26.3861
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,123.170,N/A,0.513233,42.5743,0.213254,42.5743,0.116573,45.9406,0.0886441,77.6238,52.1782
501,ZBH,176.240,N/A,0.589102,48.9109,0.177352,37.4257,0.0777432,32.2772,0.0775617,70.297,47.2277
502,ZBRA,513.999,N/A,1.54902,91.4851,0.761891,90.6931,0.258836,88.3168,0.085047,75.0495,86.3861
503,ZION,56.610,N/A,1.19898,83.1683,0.833672,94.4554,0.149699,61.9802,-0.00498441,15.6436,63.8119


In [9]:
df.sort_values('HQM Score',ascending=False,inplace=True)

df.reset_index(inplace=True,drop=True)

df=df[:50]

In [14]:
position_size = float(portfolio_size)/len(df.index)

for row in df.index:
    df.loc[row,'Number of Shares to Buy']= math.floor(position_size/df['Price'][row])
    
df.head()

/Users/User2/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,LB,67.97,294,4.54404,100,1.03231,98.2178,0.501409,99.604,0.110116,88.7129,96.6337
1,MHK,210.00,95,1.77683,94.4554,0.963306,97.4257,0.382474,97.4257,0.121838,92.2772,95.396
2,AMAT,135.46,147,1.65371,93.0693,1.13683,98.4158,0.303706,93.4653,0.133282,93.8614,94.703
3,NUE,80.89,247,1.36078,87.3267,0.660261,85.7426,0.452353,99.4059,0.161339,97.8218,92.5743
4,FCX,38.76,515,4.21759,99.802,1.27369,98.6139,0.264383,89.1089,0.096496,80.7921,92.0792


In [13]:
writer = pd.ExcelWriter('/Users/User2/Documents/freecode camp algo trading/high quality momentum strategy.xlsx',engine='xlsxwriter')
df.to_excel(writer,sheet_name='abc',index=False)    

writer.save()

